# Projeto 3: Classificação binária brest cancer com tuning dos parâmetros

## Etapa 1: Importação das bibliotecas

In [1]:
#!pip install skorch

In [2]:
import pandas as pd
import numpy as np
import torch.nn as nn
from skorch import NeuralNetBinaryClassifier
import torch
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV
torch.__version__

'1.6.0'

## Etapa 2: Base de dados

In [3]:
np.random.seed(123)
torch.manual_seed(123)

In [5]:
previsores = pd.read_csv('../Bases/entradas_breast.csv')
classe = pd.read_csv('../Bases/saidas_breast.csv')

In [6]:
previsores = np.array(previsores, dtype = 'float32')
classe = np.array(classe, dtype = 'float32').squeeze(1)

In [7]:
previsores.shape

(569, 30)

In [8]:
classe.shape

(569,)

## Etapa 3: Classe para estrutura da rede neural

In [9]:
class classificador_torch(nn.Module):
  def __init__(self, activation, neurons, initializer):
    super().__init__()
    # 30 -> 16 -> 16 -> 1
    self.dense0 = nn.Linear(30, neurons)
    initializer(self.dense0.weight)
    self.activation0 = activation
    self.dense1 = nn.Linear(neurons, neurons)
    initializer(self.dense1.weight)
    self.activation1 = activation
    self.dense2 = nn.Linear(neurons, 1)
    initializer(self.dense2.weight)
    self.output = nn.Sigmoid()

  def forward(self, X):
    X = self.dense0(X)
    X = self.activation0(X)
    X = self.dense1(X)
    X = self.activation1(X)
    X = self.dense2(X)
    X = self.output(X)
    return X

## Etapa 4: Skorch

In [10]:
classificador_sklearn = NeuralNetBinaryClassifier(module=classificador_torch,
                                                  lr = 0.001,
                                                  optimizer__weight_decay = 0.0001,
                                                  train_split=False)

## Etapa 5: Tuning dos parâmetros

In [11]:
params = {'batch_size': [10],
          'max_epochs': [100],
          'optimizer': [torch.optim.Adam, torch.optim.SGD],
          'criterion': [torch.nn.BCELoss], #, torch.nn.HingeEmbeddingLoss],
          'module__activation': [F.relu, F.tanh],
          'module__neurons': [8, 16], 
          'module__initializer': [torch.nn.init.uniform]} # _, torch.nn.init.normal_]}

In [12]:
params

{'batch_size': [10],
 'max_epochs': [100],
 'optimizer': [torch.optim.adam.Adam, torch.optim.sgd.SGD],
 'criterion': [torch.nn.modules.loss.BCELoss],
 'module__activation': [<function torch.nn.functional.relu(input, inplace=False)>,
  <function torch.nn.functional.tanh(input)>],
 'module__neurons': [8, 16],
 'module__initializer': [<function torch.nn.init._make_deprecate.<locals>.deprecated_init(*args, **kwargs)>]}

In [13]:
grid_search = GridSearchCV(estimator=classificador_sklearn, param_grid=params,
                           scoring = 'accuracy', cv = 2)
grid_search = grid_search.fit(previsores, classe)

<ipython-input-9-2a6a126076fc>:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense0.weight)
<ipython-input-9-2a6a126076fc>:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense1.weight)
<ipython-input-9-2a6a126076fc>:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense2.weight)


  epoch    train_loss     dur
-------  ------------  ------
      1       37.3239  0.4285
      2       37.3239  0.0259
      3       37.3239  0.0250
      4       37.3239  0.0269
      5       37.3239  0.0249
      6       37.3239  0.0262
      7       37.3239  0.0259
      8       37.3239  0.0250
      9       37.3239  0.0260
     10       37.3239  0.0249
     11       37.3239  0.0250
     12       37.3239  0.0249
     13       37.3239  0.0259
     14       37.3239  0.0260
     15       37.3239  0.0248
     16       37.3239  0.0259
     17       37.3239  0.0269
     18       37.3239  0.0259
     19       37.3239  0.0249
     20       37.3239  0.0249
     21       37.3239  0.0259
     22       37.3239  0.0250
     23       37.3239  0.0250
     24       37.3239  0.0260
     25       37.3239  0.0249
     26       37.3239  0.0255
     27       37.3239  0.0259
     28       37.3239  0.0269
     29       37.3239  0.0250
     30       37.3239  0.0249
     31       37.3239  0.0259
     32   

IndexError: too many indices for array

In [ ]:
melhores_parametros = grid_search.best_params_
melhor_precisao = grid_search.best_score_

In [ ]:
melhores_parametros

{'batch_size': 10,
 'criterion': torch.nn.modules.loss.BCELoss,
 'max_epochs': 100,
 'module__activation': <function torch.nn.functional.relu>,
 'module__initializer': <function torch.nn.init._make_deprecate.<locals>.deprecated_init>,
 'module__neurons': 8,
 'optimizer': torch.optim.adam.Adam}

In [ ]:
melhor_precisao

0.7524153694094391